In [3]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [4]:
import codecs

with codecs.open('news/NewsArticles.csv', 'r', 'utf-8', 'ignore') as f:
    df = pd.read_csv(f)

df = df.iloc[:,0:6]

In [5]:
# 2017/2/2（10本）の記事からなるテストデータを作成
df = df[df['publish_date'] == '2017/2/2']
len(df)

10

In [16]:
#　センテンスのリストを作る
sentence_list = []
for content in df['text']:
    if type(content) is str:
        sentences = tokenize.sent_tokenize(content)
        for sentence in sentences:
            sentence_list.append(sentence)
    else:
        pass

In [17]:
# OpenIEにかける
from pyopenie import OpenIE5
extractor = OpenIE5('http://localhost:8000')

extractions_list = []
for sentence in sentence_list:
    try:
        extractions = extractor.extract(sentence)
        extractions_list.append(extractions)
    except Exception:
        pass

In [18]:
rows = []
for extractions in extractions_list:
    if extractions == []:
        pass
    else:
        for extraction in extractions:
            sentence = extraction['sentence']
            confidence = extraction['confidence']
            arg1 = extraction['extraction']['arg1']['text']
            rel = extraction['extraction']['rel']['text']
            arg2s_list = []
            for arg2 in extraction['extraction']['arg2s']:
                arg2s_list.append(arg2['text'])
            arg2s = ' '.join(map(str, arg2s_list))
            row = [sentence, arg1, rel, arg2s, confidence]
            rows.append(row)

In [20]:
kb_df = pd.DataFrame(rows, columns = ['sentence', 'arg1', 'rel', 'arg2s', 'confidence'])
kb_df.head()

,sentence,arg1,rel,arg2s,confidence
0,"Although slavery is outlawed in every country, criminals earn more than $150 Billion every year from enslaving people.",criminals,earn,more than $ 150 billion every year,0.999000
1,"Although slavery is outlawed in every country, criminals earn more than $150 Billion every year from enslaving people.",criminals,earn,more than $150 Billion every year from enslaving people,0.894243
2,"Although slavery is outlawed in every country, criminals earn more than $150 Billion every year from enslaving people.",slavery,is outlawed,in every country,0.894243
3,But you can do something about it.,you,can do,something about it,0.464930
4,"On MARCH 14, 2017, we want YOU and your school to join us for #MyFreedomDay.",your school,to join,us for # MyFreedomDay,0.469945


In [21]:
len(kb_df)

786

In [22]:
from db import connect
engine = connect()
kb_df.to_sql(name='news',con=engine,if_exists='replace',index=None)